In [1]:
# !pip install fbm

In [2]:
import os
import pandas as pd
import numpy as np

os.chdir("/dss/dsshome1/02/ge85rik2/MarketGenerators")
from src.evaluate import metrics as m
from src.visualization.plot_option_results import OptionPricingVisualization 
from src.evaluate.option_pricing import OptionPricingEngine
from src.data.make_dataset import DataLoader

In [3]:
# capture all outputs of print functions
from io import StringIO 
import sys

class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio # free up some memory
        sys.stdout = self._stdout

In [9]:
# Define the base directory
base_dir = "numerical_results_Liao"

# Define the folders of interest
model_folders = ["GBM", "Kou_Jump_Diffusion"]

# Define the target subfolders
target_subfolders = ["CWGAN", "GMMN", "RCGAN", "RCWGAN", "SigCWGAN", "TimeGAN"]
freq = "Y"
n_in = "n-in=150"+freq
num_columns = 252

# Parameters
S0 = 1
T = 1
t = 0
K_grid = np.linspace(0.8, 1.2, 100) if freq == "M" else np.linspace(0.5, 1.5, 100)
# initialize epmty last specification
last_spec=""

# Loop through each model folder
for model_folder in model_folders:
    print(f"Start evaluating all {model_folder}-based models...")
    model_path = os.path.join(base_dir, model_folder)
    
    # Traverse the directory tree
    for root, dirs, files in os.walk(model_path):
        gen_model = os.path.basename(root)  
        
        if gen_model in target_subfolders:
            # Read the relevant npy files
            generated_file = os.path.join(root, "generated_returns_rescaled.npy")
            input_file = os.path.join(root, "input_returns_unscaled.npy")
            print(root)
            if os.path.exists(generated_file) and os.path.exists(input_file):
                input_returns = np.load(input_file).reshape((-1, num_columns))
                input_prices_np = np.exp(np.cumsum(input_returns, axis=1))     
                input_prices_df = pd.DataFrame(np.insert(input_prices_np, 0, 1, axis=1))
                
                #take the same number of output paths as the input paths
                n_input_returns = np.prod(input_returns.shape)
                generated_returns = np.load(generated_file).flatten()[:n_input_returns].reshape((-1, num_columns))
                generated_prices_np = np.exp(np.cumsum(generated_returns, axis=1))               
                generated_prices_df = pd.DataFrame(np.insert(generated_prices_np, 0, 1, axis=1))
                
                input_model = root.split("/")[1] 
                model_spec = root.split("/")[2]
                # TimeGAN is first folder ==> New evaluation
                if gen_model == "TimeGAN":
                    closePlots = False
                    # Reset output string
                    summary_output = ""
                    model_desc = "/".join(root.split("/")[1:5])
                    pairs = model_spec.split("_")
                    params = {key: float(value) for key, value in (pair.split("=") for pair in pairs)}
                    print(f"   Evaluating model {model_desc}...")

                    if input_model != "GBM" and model_spec != last_spec:
                        # create data for approximate exact price (100,000 paths) in the first run
                        # fix lambda naming:
                        if "lambda" in params:
                            params["lambda_"] = params.pop("lambda")
                        n_approx = 1000000
                        params["n"] = n_approx
                        params["T"] = T
                        params["n_points"] = num_columns+1
                        params["S0"] = S0
                        dataloader = DataLoader(method=root.split("/")[1], params=params, seed=314)
                        print(f"      ...Generating {n_approx} paths of {input_model} model...")
                        approx_df = dataloader.create_dataset(output_type="DataFrame")
                        last_spec = model_spec
                    
                    # initialize new european option pricing engine
                    european_engine = OptionPricingEngine(
                        type="European", 
                        S0=S0, 
                        T=T, 
                        t=t, 
                        ground_paths_df=input_prices_df, 
                        gen_paths_df=generated_prices_df,
                        approx_exact=(input_model != "GBM"),
                    )
                    
                    
                input_spec = "-".join(root.split("/")[1:5]).replace(".", ",")
                
                # Get respective path metrics
                if input_model == "GBM":
                    # To do: read the sigma and mu values dynamically (not important for now)
                    approx_df = None
                    european_engine.sigma = params["sigma"]
                    european_engine.r = params["mu"]
                    exact_label = "Theoretical "
                    with Capturing(summary_output) as summary_output:
                        print(gen_model + " " + input_spec)
                        m.print_basic_gbm_metrics(
                            n_periods = T, 
                            annualization_factor = 252, 
                            ground_paths_df = input_prices_df, 
                            recovered_paths_df = generated_prices_df, 
                            exp_stdev = european_engine.sigma,  
                            mu = european_engine.r,  
                            return_threshold = 0.03
                        )
                else:
                    exact_label = "Approximated "
                    with Capturing(summary_output) as summary_output:
                        print(gen_model + " " + input_spec)
                        european_engine.r = m.print_basic_non_gbm_metrics( 
                            n_periods=T,
                            annualization_factor = 252, 
                            ground_paths_df = input_prices_df, 
                            recovered_paths_df = generated_prices_df, 
                            approx_df=approx_df,
                            return_threshold = 0.03
                        )
                        
            
                print(f"         Calculating option prices for {gen_model} paths")
                # calculate all values (option prices & deviations) for different strike prices (K values)
                european_engine.gen_paths_df=generated_prices_df
                european_engine.calc_all_K(K_grid, approx_df=approx_df, recalculate_all=False)
                # Plot all types of plots
                if gen_model == "TimeGAN":
                    # initialize new plotter
                    european_plotter = OptionPricingVisualization(european_engine, exact_label=exact_label, file_name=input_spec)
                
                # use new pricing engine for respective model
                european_plotter.pe = european_engine
                
                # Save all results to files
                if gen_model == "RCWGAN":
                    # RCWGAN is last model and thus finishes the output
                    # Specify file location
                    relevant_dir = "/".join(root.split("/")[:-1])
                    print(f"      Writing summary file at {relevant_dir}.")
                    with open(f"{relevant_dir}/basic_metrics.txt", "w") as text_file:
                        text_file.write("\n".join(summary_output))
                    closePlots = True
                
                # Plot option results
                european_plotter.plot_option_prices(close=closePlots, label=gen_model)
                european_plotter.plot_option_price_deviation(close=closePlots, label=gen_model)
                european_plotter.plot_option_price_deviation_relative(close=closePlots, label=gen_model, zoom_ylimits=(0,50))
            else:
                print("No file found yet.")

print("Done.")

Start evaluating all GBM-based models...
numerical_results_Liao/GBM/mu=0.05_sigma=0.2/n-in=1000Y/seed=42/SigCWGAN
No file found yet.
numerical_results_Liao/GBM/mu=0.05_sigma=0.2/n-in=150Y/seed=42/TimeGAN
   Evaluating model GBM/mu=0.05_sigma=0.2/n-in=150Y/seed=42...
         Calculating option prices for TimeGAN paths
numerical_results_Liao/GBM/mu=0.05_sigma=0.2/n-in=150Y/seed=42/CWGAN
         Calculating option prices for CWGAN paths
numerical_results_Liao/GBM/mu=0.05_sigma=0.2/n-in=150Y/seed=42/SigCWGAN
         Calculating option prices for SigCWGAN paths
numerical_results_Liao/GBM/mu=0.05_sigma=0.2/n-in=150Y/seed=42/RCGAN
         Calculating option prices for RCGAN paths
numerical_results_Liao/GBM/mu=0.05_sigma=0.2/n-in=150Y/seed=42/GMMN
         Calculating option prices for GMMN paths
numerical_results_Liao/GBM/mu=0.05_sigma=0.2/n-in=150Y/seed=42/RCWGAN
         Calculating option prices for RCWGAN paths
      Writing summary file at numerical_results_Liao/GBM/mu=0.05_sigma=0.2

KeyboardInterrupt: 